# imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install unidecode
!pip install emoji

import re
import warnings
import numpy as np
import pandas as pd
import tensorflow as tf
from string import digits
import json
from datetime import datetime

import math
import pickle
from collections import Counter
import matplotlib.pyplot as plt
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
from nltk import download as nltk_download
from emoji import demojize
import emoji
import unidecode as ud
from nltk import bigrams

warnings.filterwarnings('ignore')
nltk_download('stopwords')
nltk_download('punkt')
nltk_download('wordnet')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 4.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 4.3 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=36fe36f35c0068b9823b0395e0796d4d3542e47267a292af55a784bd1487213b
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

# data gathering

In [ ]:
path = 'drive/MyDrive/crypto-classifier/data'
d1=pd.read_csv(f'{path}/new_tag/Darya.v1.csv', encoding='UTF-8',dtype='str')
d2=pd.read_csv(f'{path}/tag3/Nargess.v2.csv', encoding='UTF-8',dtype='str')
d3=pd.read_csv(f'{path}/tag3/Hana.v2.csv', encoding='UTF-8',dtype='str')
d4=pd.read_csv(f'{path}/multy_platform/instagram_tags_1.csv', encoding='UTF-8',dtype='str')
d5=pd.read_csv(f'{path}/multy_platform/Reddit_3000_Data_tagged_1.csv', encoding='UTF-8',dtype='str')

In [ ]:
data=pd.concat([d4.rename(columns={'Polarity'	:'polarity'}),d5]).reset_index(drop=True)
data['polarity']=data['polarity'].astype(float).astype(int)
labels = {0:'not positive', 1:'positive'}
for i in range(len(data['polarity'])):
  data['polarity'][i]=labels[data['polarity'][i]]
data=data[['text','polarity']]

In [ ]:
data2=data

In [ ]:
d2=d2.iloc[:400,:-2]
data=pd.concat([d1,d2])
data=pd.concat([data,d3])
data=data.rename(columns={'Unnamed: 0.1'	:'index1'})
data=data.rename(columns={'Unnamed: 0'	:'index0'})
data=data[data['ns'].isna()].reset_index(drop=True)
data=data.dropna(subset=['concat_fields']).reset_index(drop=True)
# data

In [ ]:
# make Uniformity labels
data['signal'] = data['signal'].fillna('no signal').apply(lambda x:x.lower())
data['polarity'] = data['polarity'].fillna('neutral').apply(lambda x:x.lower())

data['signal']=data['signal'].apply(lambda x:'no signal' if x in ['no siganl','no signale','not buy','-','bno signal'] else x)
data['signal']=data['signal'].apply(lambda x:'hold' if x in ['bhold','keep'] else x)
data['signal']=data['signal'].apply(lambda x:'buy' if x in ['nbuy','positie','positive'] else x)
data['polarity']=data['polarity'].apply(lambda x:'neutral' if x in ['nutral','-'] else x)
data['polarity']=data['polarity'].apply(lambda x:'negative' if x in ['negetive','negetive\n'] else x)
data['polarity']=data['polarity'].apply(lambda x:'positive' if x in ['positve','positve','positie','posiitve','postive'] else x)
print('categories: ',Counter(data['signal']))
print('polarity: ',Counter(data['polarity']))

categories:  Counter({'buy': 1089, 'no signal': 716, 'sell': 101, 'hold': 21})
polarity:  Counter({'positive': 1265, 'neutral': 483, 'negative': 179})


In [ ]:
# data[['concat_fields','polarity']].rename(columns={'concat_fields'	:'text'})
data = pd.concat([data2.rename(columns={'text':'concat_fields'}),data[['concat_fields','polarity']]]).reset_index(drop=True)

# columns
['index0', 'index1', 'contributors', 'coordinates', 'created_at',
'display_text_range', 'entities', 'favorite_count', 'favorited',
'full_text', 'geo', 'id', 'id_str', 'in_reply_to_screen_name',
'in_reply_to_status_id', 'in_reply_to_status_id_str',
'in_reply_to_user_id', 'in_reply_to_user_id_str', 'is_quote_status',
'lang', 'place', 'retweet_count', 'retweeted', 'source', 'truncated',
'user', 'extended_entities', 'possibly_sensitive', 'retweeted_status',
'quoted_status', 'quoted_status_id', 'quoted_status_id_str',
'quoted_status_permalink', 'user_sname', 'withheld_copyright',
'withheld_in_countries', 'withheld_scope', 'concat_fields',
'clean_tweet', 'coins', 'coins2', 'coins3', 'signal', 'polarity',
'Ambiguous', 'ns', 'discotion or ad']

---



---



# media type

In [ ]:
import json
data['has_media']=data['extended_entities'].notnull()
tm=[]
for i,d in enumerate(data['extended_entities']):
  try:
    if data['has_media'][i]:
      tm.append(json.loads(d.replace('\'','\"').replace(' False',' \"False\"').replace(' True',' \"True\"').replace(' None',' \"None\"'))['media'][0]['type'])
    else:
      tm.append('0')
  except:
    print(i)
    tm.append(-1)
data['media_type']=tm
print(Counter(data['media_type']))

media2integer={'0': 0, 'animated_gif': 1, 'photo': 2, 'video': 3 , -1: 0}

from sklearn.preprocessing import OneHotEncoder
data['media_type_int']=data['media_type'].apply(lambda x : media2integer[x])
onehot_encoder = OneHotEncoder(sparse=False)

data[['no_media','animated_gif', 'photo','video']]=onehot_encoder.fit_transform(data[['media_type_int']]).astype(int)

1491
Counter({'0': 1387, 'photo': 434, 'video': 65, 'animated_gif': 40, -1: 1})


# links

In [ ]:
normalizer= lambda x:x.replace('\"','\";"').replace('\'','\"').replace('\";"','\'').replace(' False',' \"False\"').replace(' True',' \"True\"').replace(' None',' \"None\"')

links=[]
quoted_status = data['quoted_status'].notna()
retweeted_status = data['retweeted_status'].notna()
for i in range(len(data)):
  try:
    if quoted_status[i]:
      links.append([j['expanded_url'] for j in json.loads(normalizer(data.iloc[i]['quoted_status']))['entities']['urls']]+[j['expanded_url'] for j in json.loads(normalizer(data.iloc[i]['entities']))['urls']])
    elif retweeted_status[i]:
      links.append([j['expanded_url'] for j in json.loads(normalizer(data.iloc[i]['retweeted_status']))['entities']['urls']])
    else:
      links.append([j['expanded_url'] for j in json.loads(normalizer(data.iloc[i]['entities']))['urls']])
  except:
    # print(i)
    links.append([])
    pass
data['links']=np.array(links)

# preprocess

In [ ]:

def extract_emojis(s):
  return [c for c in s if c in emoji.UNICODE_EMOJI['en']]

""" Normalize text """
def normalizeToken(token):
    """return normal tweet with mentions and http url
    Args:
        token ([string]): [a word(a token)]
    Returns:
        [string]: [normalized token]
    """
    if token.startswith("@"):
        return "@user"
    elif token.lower().startswith("http") or token.lower().startswith("www"):
        return "httpurl"
    else:
        return (token)

def normalizeTweets(tweets):
    """function for preprocessing
        - demojize
        - unidecode
        - lower
        - lammetize

    Args:
        tweets ([numpy list]): [list of plane text tweets]

    Returns:
        [numpy list]: [list of normalized tweets]
    """

    f = open(f"{path}/englishstop.txt", "r")
    stop_wl_1=f.read()
    f = open(f"{path}/stop_words_english.txt", "r")
    stop_wl_2=f.read()

    punctuations=r'''"&()*,.:;?[\]^`{|}~'''

    # # replace emojies
    tweets=tweets.apply(lambda x:" ".join(demojize(x, delimiters=(" ", " ")).split()))

    # # delete numbers and punctuations

    tweets = tweets.apply(lambda x: ud.unidecode(x))
    tweets = tweets.apply(lambda x: x.translate(str.maketrans(' ', ' ', punctuations)))
    # #lower
    tweets = tweets.apply(lambda x: x.lower())
    # # normalize
    lemma = WordNetLemmatizer()
    # normTweets = tweets.apply(lambda x: (" ".join([normalizeToken(token) for token in x.split()])))
    normTweets = tweets.apply(lambda x: (" ".join([lemma.lemmatize(token, pos = "v") for token in x.split()])))
    normTweets = normTweets.apply(lambda x: (" ".join([lemma.lemmatize(token, pos = "n") for token in x.split()])))

    # # remove stopwords
    normTweets = normTweets.apply(lambda x: " ".join([word for word in x.split() if word not in stopwords.words("english")]))
    normTweets = normTweets.apply(lambda x: " ".join([word for word in x.split() if word not in stop_wl_1]))
    # normTweets = normTweets.apply(lambda x: " ".join([word for word in x.split() if word not in stop_wl_2]))

    for i in range(len(normTweets)):
        if not len(normTweets[i]):
             normTweets[i]='nan'
    return normTweets

def unitext(tweets):
    dolars = tweets.apply(lambda x:set([ t.replace('$','') for t in x.split() if t.startswith('$')and len(t)>1 ]))

    tweets = tweets.apply(lambda x: (" ".join([normalizeToken(token) for token in x.split()])))
    tweets = tweets.apply(lambda x: re.sub(r"\d+", " a_number ", str(x)))
    ut=[]
    for i in range(len(tweets)):
      ut.append(unicoin(tweets[i],dolars[i]))
    return np.array(ut)

def extract_features(mydata):

  """extract hashta """
  mydata['hashtags'] = mydata['clean_text'].str.findall("(#[^#\s]+)")
  mydata['dolar'] = mydata['clean_text'].apply(lambda x:set([ t.replace('$','') for t in x.split() if t.startswith('$')and len(t)>1 ]))
  print('hashtag extraction done!')

  mydata['mentions']=mydata['concat_fields'].str.findall("(@[^@\s]+)")
  print('mention extraction done!')

  mydata['emojies'] = mydata['concat_fields'].apply(lambda x: extract_emojis(x))

  # data['links']=data['concat_fields'].apply(lambda x: [token  for token in x.split() if token.lower().startswith("http") or token.lower().startswith("www")])
  print('link extraction done!')

  mydata['tokenied'] = mydata['clean_text'].apply(lambda x:x.split(' '))
  print('tokenizing done!')

  mydata['bigrams'] = mydata['tokenied'].apply(lambda x: list(bigrams(x)))
  mydata['bigrams'] = mydata['bigrams'].apply(lambda x: [str(tupl[0] + ' ' + tupl[1]) for tupl in x])
  print('bigram extraction done!')


  mydata['num_hashtags']=mydata['hashtags'].apply(lambda x: len(x))
  mydata['num_mentions']=mydata['mentions'].apply(lambda x: len(x) )

  # NYSE=(UTC -5) 2:30 p.m. to 9 p.m

  mydata['created_at']=mydata['created_at'].fillna('2000-10-10 24:00:00+00:00')
  mydata['Newyork_stockmarket_time']=False
  mydata['Newyork_stockmarket_date']=False
  mydata['day_week']=-1
  mydata['day_month']=-1
  mydata['month']=-1

  for i in range(len(mydata)):
    x=mydata['created_at'][i]
    try:
      mydata['Newyork_stockmarket_time'][i] = True if x.split()[1].split('+')[0]>'14:30:00' and x.split()[1].split('+')[0]< '21:00:00' else False
      mydata['Newyork_stockmarket_date'][i] = pd.Timestamp(x.split()[0]).dayofweek<5 if x.split()[0]!='2000-10-10'  else False
      data['day_week'][i]=pd.Timestamp(x.split()[0]).dayofweek if x.split()[0]!='2000-10-10' else -1
      data['day_month'][i]=x.split()[0].split('-')[2] if x.split()[0]!='2000-10-10'  else -1
      data['month'][i]=x.split()[0].split('-')[1] if x.split()[0]!='2000-10-10'  else -1

    except:
      print(i,x)
  # mydata['Newyork_stockmarket_time']=mydata['created_at'].fillna('2000-10-10 24:00:00+00:00').apply(lambda x:True if x.split()[1].split('+')[0]>'14:30:00' and x.split()[1].split('+')[0]< '21:00:00' else False).astype(int)
  # mydata['Newyork_stockmarket_date']=mydata['created_at'].fillna('0000-00-00 24:00:00+00:00').apply(lambda x:pd.Timestamp(x.split()[0]).dayofweek<5 if x.split()[0]!='2000-10-10'  else False ).astype(int)
  mydata['Newyork_stockmarket']=mydata['Newyork_stockmarket_time']*mydata['Newyork_stockmarket_date'].astype(int)


  print(' successfull Newyork_stockmarket_time ')

  # day of week
  # data['day_week']=data['created_at'].fillna('0000-00-00 24:00:00+00:00').apply(lambda x:pd.Timestamp(x.split()[0]).dayofweek if x.split()[0]!='2000-10-10'  else -1 )
  # data['day_month']=data['created_at'].fillna('0000-00-00 24:00:00+00:00').apply(lambda x:x.split()[0].split('-')[2] if x.split()[0]!='2000-10-10'  else -1 ).astype('int')
  # data['month']=data['created_at'].fillna('0000-00-00 24:00:00+00:00').apply(lambda x:x.split()[0].split('-')[1] if x.split()[0]!='2000-10-10'  else -1 ).astype('int')

  return mydata

def unicoin(tweet,dolar):
  tokens=tweet.split()
  for i,w in enumerate(tokens):
    if w[0]=='$' or w[0]=='#':
      w=w[1:]

    if w in dolar or w in list(coins['coin_name']) or w in list(coins['symbol']):
      tokens[i]='a_coin_name'

  return ' '.join(tokens)

coins=pd.read_csv(f'{path}/Top_100_coinMarketCap.csv')
coins['with$']=coins['symbol'].apply(lambda x:'$'+x+' ')
coins['with#']=coins['symbol'].apply(lambda x:'#'+x+' ')
coins['symbol']=coins['symbol'].apply(lambda x:' '+x.upper()+' ')
coins


,cname,coin_name,symbol,with$,with#
0,1inch network,1inch network,1INCH,$1inch,#1inch
1,aave,aave,AAVE,$aave,#aave
2,algorand,algorand,ALGO,$algo,#algo
3,amp,amp,AMP,$amp,#amp
4,arweave,arweave,AR,$ar,#ar
...,...,...,...,...,...
95,wrapped bitcoin,wrapped bitcoin,WBTC,$wbtc,#wbtc
96,xdc network,xdc network,XDC,$xdc,#xdc
97,xrp,xrp,XRP,$xrp,#xrp
98,yearn.finance,yearn.finance,YFI,$yfi,#yfi


In [ ]:
data['concat_fields'] = data['concat_fields'].apply(lambda x: x.replace('&amp;','and'))
data['clean_text']=normalizeTweets(data['concat_fields'])
data['unicode_text_clean']=unitext(data['clean_text'])
data['unicode_text_concat']=unitext(data['concat_fields'])

data=extract_features(data)

hashtag extraction done!
mention extraction done!
link extraction done!
tokenizing done!
bigram extraction done!
1240 1.47E+18
1270 1.50E+18
1307 1.41E+18
1315 1.39E+18
1330 1.45E+18
1373 1.47E+18
1414 1.44E+18
1433 1.44E+18
1465 1.36E+18
1487 1.45E+18
1551 1.41E+18
1562 1.50E+18
1572 1.40E+18
1586 1.47E+18
1610 1.44E+18
1633 1.41E+18
1711 1.47E+18
1730 1.46E+18
1747 1.46E+18
1780 1.44E+18
1781 1.35E+18
1849 1.47E+18
1916 1.47E+18
1920 1.41E+18
 successfull Newyork_stockmarket_time 


In [ ]:
data['no_punc_clean_tweet'] = data['clean_text'].apply(lambda x: re.sub('[^a-zA-Z0-9#$]+\s*', ' ',x))
data['no_punc_concat_fields'] = data['concat_fields'].apply(lambda x: re.sub('[^a-zA-Z0-9#$]+\s*', ' ',x))
# data['clean_tweet'] = data['clean_text'].apply(lambda x: x.replace('&amp;','and'))
data['coins'] = data['no_punc_clean_tweet'].apply(lambda x :  [coins['cname'][i] for i in range(len(coins)) if (coins['coin_name'][i]+' ' in x.lower()) or (coins['symbol'][i] in x) or (coins['with$'][i] in x.lower()) or (coins['with#'][i] in x.lower())] )
# print('The number of tweets that has coin names is :',len(clean_main_df))
# clean_main_df

for i in range(len(data['coins'])):
  if ' btc ' in data.iloc[i]['no_punc_clean_tweet'] and 'bitcoin' not in data.iloc[i]['coins'] :
    data.iloc[i]['coins'].append('bitcoin')

# clean_main_df2=data[data['coins'].apply(lambda x:len(x)!=0)]
data['coins2'] = data['no_punc_clean_tweet'].apply(lambda x :  [coins['cname'][i] for i in range(len(coins)) if (' '+coins['coin_name'][i]+' ' in x.lower()) or ('$'+coins['coin_name'][i]+' 'in x.lower()) or ('#'+coins['coin_name'][i]+' ' in x.lower())] )
data['coins3'] = data['no_punc_concat_fields'].apply(lambda x :  [coins['cname'][i] for i in range(len(coins)) if (coins['symbol'][i] in x)                    or (coins['with$'][i] in x.lower())            or (coins['with#'][i] in x.lower())] )

# print('The number of tweets that has coin names is :',len(data))

In [ ]:
data['coins1']='A'
for i in range(len(data)):
  try:
    data['coins1'][i]=  list(set(data['coins2'][i]+data['coins3'][i]))
  except:
    print(i,data['coins2'][i])
data [['coins2','coins3','coins1','coins']]
data['coins']=data['coins1']
data.drop(columns=['coins2','coins3','coins1','no_punc_clean_tweet','no_punc_concat_fields'])

,index0,index1,contributors,coordinates,created_at,display_text_range,entities,favorite_count,favorited,full_text,...,bigrams,num_hashtags,num_mentions,emojies,Newyork_stockmarket_time,Newyork_stockmarket_date,Newyork_stockmarket,day_week,day_month,month
0,NaN,NaN,NaN,NaN,2000-10-10 24:00:00+00:00,NaN,NaN,NaN,NaN,NaN,...,"[core philosophy, philosophy revolve, revolve ...",2,1,[👇],False,False,0,-1,-1,-1
1,NaN,NaN,NaN,NaN,2000-10-10 24:00:00+00:00,NaN,NaN,NaN,NaN,NaN,...,"[today's bitcoin, bitcoin upsurge, upsurge coi...",0,0,[],False,False,0,-1,-1,-1
2,234,250964,NaN,NaN,2020-02-11 23:53:19+00:00,"[0, 139]","{'hashtags': [], 'symbols': [], 'urls': [], 'u...",0,0,RT @PeterMcCormack: *Bitcoin Around the World*...,...,"[bitcoin world, world wbd193, wbd193 -, - colo...",0,0,[],False,True,0,1,11,2
3,576,137938,NaN,NaN,2020-03-17 03:55:01+00:00,"[0, 139]","{'hashtags': [{'indices': [66, 74], 'text': 'B...",0,0,RT @longhashdata: High leverage in the crypto ...,...,"[high leverage, leverage crypto, crypto market...",1,3,[],False,True,0,1,17,3
4,651,137926,NaN,NaN,2020-03-24 08:25:41+00:00,"[0, 213]","{'hashtags': [], 'symbols': [], 'urls': [], 'u...",216,0,Fed printing unlimited money to pump markets =...,...,"[feed print, print unlimited, unlimited money,...",0,0,[],False,True,0,1,24,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1922,1037,101371,NaN,NaN,2022-03-03 19:22:28+00:00,"[0, 125]","{'hashtags': [], 'symbols': [], 'urls': [{'dis...",15,FALSE,4b/ No Ledn customers were affected. @cryptono...,...,"[simon dixon, dixon mauricio, mauricio bartlol...",0,1,[],True,True,1,3,3,3
1923,1183,98633,NaN,NaN,2022-03-04 13:35:06+00:00,"[0, 144]","{'hashtags': [{'indices': [77, 85], 'text': 'B...",0,FALSE,RT @dinshoo12345: 🇺🇦 The Ukrainian government ...,...,"[ukraine ukrainian, ukrainian government, gove...",3,0,[🤨],False,True,0,4,4,3
1924,1398,246043,NaN,NaN,2022-03-06 02:39:16+00:00,"[0, 107]","{'hashtags': [{'indices': [56, 64], 'text': 'B...",3457,FALSE,JUST IN: The number of addresses holding at le...,...,"[number address, address hold, hold 1000, 1000...",1,0,[],False,False,0,6,6,3
1925,1345,250982,NaN,NaN,2022-03-06 07:26:57+00:00,"[0, 119]","{'hashtags': [], 'media': [{'display_url': 'pi...",36,FALSE,Conditions ‘ripe’ for Bitcoin adoption amid ge...,...,"[condition 'ripe', 'ripe' bitcoin, bitcoin ado...",0,1,[],False,False,0,6,6,3


In [ ]:
data['has_media']=data['has_media'].astype(int)

# save

In [ ]:
data.drop(columns=['coins2','coins3','coins1'	,'no_punc_clean_tweet','no_punc_concat_fields']).to_csv(f"{path}/tag3/cleanTweet1000.csv",encoding='utf-8',index=False)
# data.to_csv("new_tag/cleanTweet1000.csv",encoding='utf-8',index=False)

In [ ]:
with open(f'{path}/tag3/cleanTweet1000.pickle', 'wb') as fp:
    pickle.dump(data.drop(columns=['coins2','coins3','coins1'	,'no_punc_clean_tweet','no_punc_concat_fields']), fp)